In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
from torchvision import transforms
from albumentations import Compose, Normalize, RandomCrop, HorizontalFlip, ShiftScaleRotate, HueSaturationValue
from albumentations.pytorch import ToTensorV2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models
from torchmetrics import AUROC
import io
import matplotlib.pyplot as plt
import time
import copy
import albumentations as A
import torch.nn.functional as F
from sklearn.model_selection import train_test_split


from glob import glob
from tqdm import tqdm

import warnings
from sklearn.metrics import f1_score,accuracy_score,recall_score,roc_auc_score,roc_curve

In [2]:
# https://www.kaggle.com/code/abdelrahmanmuhsen/animal-body-parts-segmentation

In [3]:
from transformers import SegformerModel, SegformerConfig

import torch

import torch.nn as nn

from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation

from PIL import Image

import requests

from transformers import SegformerPreTrainedModel, SegformerForSemanticSegmentation

from transformers import SegformerModel, SegformerConfig

from torch.utils.data import Dataset, DataLoader
       

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
train_images = "/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/MICHE_MULTICLASS/Dataset/train_img/"
train_masks  = "/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/MICHE_MULTICLASS/Dataset/train_masks/"


val_images = "/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/MICHE_MULTICLASS/Dataset/val_img"

val_masks =  "/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/MICHE_MULTICLASS/Dataset/val_masks/" 
n_classes = 3
batch_size = 1

train_x = sorted(
        glob(f"{train_images}/*"))
train_y = sorted(
        glob(f"{train_masks}/*"))
valid_x = sorted(
        glob(f"{val_images}/*"))
valid_y = sorted(
        glob(f"{val_masks }/*"))

In [6]:
n_classes =3
colors = [ [  0,   0,   0],[0,255,0],[0,0,255]]
label_colours = dict(zip(range(n_classes), colors))

valid_classes = [0,85, 170]
class_names = ["Background","Pupil","Iris"]


class_map = dict(zip(valid_classes, range(len(valid_classes))))
n_classes=len(valid_classes)

id2label = {v:class_names[v] for k,v in class_map.items()}

In [7]:
img_resize = (224,224)
class Iris(Dataset):
    def __init__(self,images,masks,img_resize=img_resize,transform = None):
        self.transform = transform
        self.images = images
        self.masks = masks
        self.img_resize = img_resize
    def __len__(self):
        return len(self.images)
    def __getitem__(self,index):
        img = np.array(Image.open(self.images[index]).resize(self.img_resize)).astype("float32")
        
        img = img/255.0
        mask = np.array(Image.open(self.masks[index]).resize(self.img_resize))
        
        sample = {"image":img,"mask":mask}
        
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample

In [8]:
feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b4-finetuned-ade-512-512")


/home/nipun/mambaforge/envs/LuminEye/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/nipun/mambaforge/envs/LuminEye/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [9]:
def transform(sample):
    data_transforms = Compose([                  
                        ToTensorV2()
                    ])
    sample['image']= sample['image']
    sample = data_transforms(**sample)
    sample['image']=feature_extractor(sample['image'], return_tensors="pt")
    sample['image']=sample['image']['pixel_values']    
    return sample

In [10]:
train_data = Iris(train_x,train_y,transform = transform)
val_data  = Iris(valid_x,valid_y,transform=transform)

In [11]:
train_batch = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
test_batch = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False,drop_last=True)

In [12]:
ex_img_train = next(iter(train_batch))["image"]
ex_mask_train = next(iter(train_batch))["mask"]

print(ex_img_train.shape)
print(ex_mask_train.shape)

torch.Size([1, 1, 3, 512, 512])
torch.Size([1, 224, 224])


In [13]:
plt.figure(figsize=(15,15))
plt.imshow(ex_mask_train[1])
print(np.unique(ex_mask_train))

IndexError: index 1 is out of bounds for dimension 0 with size 1

<Figure size 1500x1500 with 0 Axes>

In [15]:
class CustomSegFormer2(SegformerPreTrainedModel):
    def __init__(self, config, in_channel, n_classes=3):
        super().__init__(config)
        self.segformer = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b4-finetuned-ade-512-512", ignore_mismatched_sizes=True)
        self.logits_outputs = nn.Conv2d(in_channel, out_channels=n_classes, kernel_size=1, stride=1, padding=0)
        

    def forward(self, pixel_values):
        # forward to get raw logits
        pixel_values=torch.squeeze(pixel_values, dim=1)
        outputs = self.segformer(pixel_values) # what's the number of output channels?
        logits_outputs = self.logits_outputs(outputs[0])
        # next, do whatever you want with the logits to compute loss
        # upsample logits here; the predicted distance map is also needed to unsampled, right?
        upsampled_logits = nn.functional.interpolate(logits_outputs, size=img_shape, mode="bilinear", align_corners=False)
        # calculate losses for logits and distance map respectively and sum the two losses
        
        return  upsampled_logits

In [16]:
from transformers import SegformerModel, SegformerConfig
import torch
import numpy as np

configuration = SegformerConfig()
model=CustomSegFormer2(configuration,in_channel=150)

In [17]:
import segmentation_models_pytorch as smp
dice_loss=smp.losses.DiceLoss('multiclass')
def iou(output, target):
    output=F.softmax(output, dim=1)
    output = torch.argmax(output, dim=1)
    tp, fp, fn, tn = smp.metrics.get_stats(output, target, mode='multiclass',num_classes=3)
    return smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")

In [18]:
def fit(epochs, model, train_loader, val_loader, criterion, optimizer):
    train_losses = []
    test_losses = []
    val_iou = []; 
    train_iou = []; 
    min_loss = np.inf
    max_val_iou_score=0
    increase = 0 ; not_improve=0

    model.to(device)
    fit_time = time.time()
    for e in range(epochs):
        since = time.time()
        running_loss = 0
        iou_score = 0
        accuracy = 0
        #training loop
        model.train()
        for i, data in enumerate(train_loader):
            #training phase
            image_tiles = data['image']
            mask_tiles = data['mask']


            image = image_tiles.to(device)
            mask = mask_tiles.type(torch.LongTensor).to(device)
            #forward
            output = model(image).to(device)
            #loss = criterion(output, mask.type(torch.LongTensor).to(device)).requires_grad_()
            loss =criterion(output, mask)
            #evaluation metrics
            iou_score += iou(output, mask)
            #backward
            loss.backward()
            optimizer.step() #update weight          
            optimizer.zero_grad() #reset gradient
            
            running_loss += loss.item()
            
            model.eval()
            test_loss = 0
            val_iou_score = 0
        #validation loop
        with torch.no_grad():

            for i, data in enumerate(val_loader):
                image_tiles = data['image']
                mask_tiles = data['mask']

                image = image_tiles.to(device); mask = mask_tiles.type(torch.LongTensor).to(device);
                output = model(image)
                #evaluation metrics
                val_iou_score +=  iou(output, mask)
                #loss
                loss = criterion(output, mask)                                  
                test_loss += loss.item()

        #calculatio mean for each batch
        train_losses.append(running_loss/len(train_loader))
        test_losses.append(test_loss/len(val_loader))


        if max_val_iou_score < (val_iou_score/len(val_loader)):
            print('val_iou_increased.. {:.3f} >> {:.3f} '.format(max_val_iou_score, (val_iou_score/len(val_loader))))
            max_val_iou_score =  (val_iou_score/len(val_loader))
            increase += 1
            not_improve=0
            if increase % 5 == 0:
                print('saving model...')
                torch.save(model, 'segformer-{:.3f}.pt'.format(val_iou_score/len(val_loader)))

        else :
            not_improve += 1
            if not_improve == 7:
                print('Loss not decrease for 7 times, Stop Training')
                break


        #iou
        val_iou.append(val_iou_score/len(val_loader))
        train_iou.append(iou_score/len(train_loader))
        print("Epoch:{}/{}..".format(e+1, epochs),
              "Train Loss: {:.3f}..".format(running_loss/len(train_loader)),
              "Val Loss: {:.3f}..".format(test_loss/len(val_loader)),
              "Train mIoU:{:.3f}..".format(iou_score/len(train_loader)),
              "Val mIoU: {:.3f}..".format(val_iou_score/len(val_loader)),
              "Time: {:.2f}m".format((time.time()-since)/60))

    history = {'train_loss' : train_losses, 'val_loss': test_losses,
               'train_miou' :train_iou, 'val_miou':val_iou,
               }
    print('Total time: {:.2f} m' .format((time.time()- fit_time)/60))
    return history

In [19]:
epoch = 5
lr = 0.00006
criterion  =dice_loss
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
history = fit(epoch, model, train_batch,test_batch, criterion, optimizer)

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 5.81 GiB total capacity; 677.46 MiB already allocated; 10.00 MiB free; 698.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF